<div class="alert alert-block alert-info">Compare the list of relevant words obtained for each gold standard summary to the list of most important words used for our tf-idf based summarisations.
    
The notebook is structured as follows:
<ol>
  <li>get the words that we based the english bible summaries on</li>
  <li>definition of similarity measure function</li>
  <li>execution of similarity measure functions on our topten words in comparisson to different goldstandard summaries</li>
    <li>Examine german summaries</li>
    <li>First steps of attempt for using wordNets</li>
</ol>

<div>

In [20]:
import re

#read the topten words used for the summaries generated by our function 

#with open('topten_words.txt', 'r') as f:
#    words=f.readlines()
with open('topten_words_summaries.txt', 'r') as f:
    words=f.readlines()    
#clean up the format 
#--> combine all the single chars one long string 
#--> split that again by ',' to obtain a list:
topten_words = []
for i in range(len(words)):
    concat_string = ''.join(words[i]) 
    cleaned_string = re.sub('\]','',re.sub('\[','',concat_string))
    topten_words.append(cleaned_string.split(','))


# remove punctuation and \n
for i in range(len(topten_words)):
    for j in range(len(topten_words[i])):
        topten_words[i][j] = re.sub(' ', '', re.sub('\'', '', topten_words[i][j].rstrip())).lower()
        
print(topten_words[1])
print(len(topten_words))


['moses', 'lord', 'aaron', 'pharaoh', 'sockets', 'israel', 'people', 'egypt', 'tabernacle', 'land']
66


In [21]:
with open('words_top_tfidf_score_summaries.txt', 'r') as f:
    words=f.readlines()    
#clean up the format 
#--> combine all the single chars one long string 
#--> split that again by ',' to obtain a list:

words_top_tfidf_scores = []
for i in range(len(words)):
    concat_string = ''.join(words[i]) 
    cleaned_string = re.sub('\]','',re.sub('\[','',concat_string))
    words_top_tfidf_scores.append(cleaned_string.split(','))

# remove punctuation and \n and make sure we have only lower case words
for i in range(len(words_top_tfidf_scores)):
    for j in range(len(words_top_tfidf_scores[i])):
        words_top_tfidf_scores[i][j] = re.sub(' ', '', re.sub('\'', '', words_top_tfidf_scores[i][j].rstrip())).lower()
        
print(words_top_tfidf_scores[0])

['joseph', 'luz', 'and', 'egyptso', 'came', 'canaan', 'land', 'went', 'bethel', 'jacob', 'people', 'name', 'what']


<div class="alert alert-block alert-info">
2. definition of similarity measure function

<div>

In [3]:
def compareLists(gold_summary_words, topten_words):
    #use a score calculation to get a educated comparisson measurement
    counter_common = 0
    diff_gold_summary_words = list(set(gold_summary_words) - set(topten_words))
    diff_topten_words = list(set(topten_words) - set(gold_summary_words))
    
    same_elem = []
    for elem in gold_summary_words:
        if elem in topten_words:
            same_elem.append(elem)

    nr_words_gold_sum =len(gold_summary_words) 
    nr_common_words = len(same_elem)
   # return same_elem, nr_common_words,nr_words_gold_sum, diff_gold_summary_words, diff_topten_words
    return same_elem, nr_common_words,nr_words_gold_sum

In [4]:
def rouge_bleu(out,ref):
    """
    Input:
        out is the machine-outputted summary as list of words
        ref is the gold_standard as a list of words
    Output:
        rouge = number_of_overlappting_words / total_words_in_reference_summary
        bleu = number_of_overlappting_words / total_words_in_system_summary
    """

    out_i = [0] * len(out)
    ref_i = [0] * len(ref)
    for o, olem in enumerate(out):
        for r, rlem in enumerate(ref):
            if olem == rlem:
                if out_i[o] == 0:
                    if ref_i[r] == 0:
                        out_i[o] = 1
                        ref_i[r] = 1
    rouge = sum(out_i) / len(ref)
    bleu = sum(out_i) / len(out)

    return rouge, bleu

<div class="alert alert-block alert-info">
3. execution of similarity measure functions on our topten words in comparisson to different goldstandard summaries
<ul>
  A) goldstandard summaries = first senteces of each book <br>
  B) goldstandard summaries = summaries for each book found online at https://www.printfriendly.com/p/g/tp79hs <br>
  C) goldstandard summaries = summaries for each chapter combined per book, found online at 
</ul>
<div>

<div class="alert alert-block alert-info">A) goldstandard summaries = first senteces of each book<div>

In [5]:
with open('gold_summaries_words.txt', 'r') as f:
    books=f.readlines()
    
gold_summaries_words = []
for i in range(len(books)):
    concat_string = ''.join(books[i]) 
    cleaned_string = re.sub('\n','',re.sub('\'','',re.sub(' ','',re.sub('\]','',re.sub('\[','',concat_string))))).lower()
    gold_summaries_words.append(cleaned_string.split(','))


#allways check the length
print(len(gold_summaries_words))

66


In [67]:
#Now we can execute the rouge_bleu function for our topten_words and the gold standard A)
scores_topten = []
for n in range(len(gold_summaries_words)):
    #gold_summaries_words[n].pop(0)
    rouge,bleu = rouge_bleu(topten_words[n], gold_summaries_words[n])
    scores_topten.append([n,rouge,bleu ])
#print(scores_topten)


In [62]:
len(scores_topten)
rouge_scores_top_ten_one_sentence = 0
bleu_scores_topten_one_sentence = 0

for elem in scores_topten:
    rouge_scores_topten_one_sentence = rouge_scores_top_ten_one_sentence+elem[1]
    bleu_scores_topten_one_sentence = bleu_scores_topten_one_sentence+elem[2]
print('ROUGE : '+str(rouge_scores_topten_one_sentence))
print('BLEU  : '+str(bleu_scores_topten_one_sentence))

ROUGE : 0.09375
BLEU  : 16.600000000000005


In [46]:
#and we can execute rouge_bleu for our top_tfidf_score words and the gold standard A)

scores_top_score = []
for n in range(len(gold_summaries_words)):
    #gold_summaries_words[n].pop(0)
    rouge,bleu = rouge_bleu(words_top_tfidf_scores[n], gold_summaries_words[n])
    scores_top_score.append([n,rouge,bleu ])
print(scores_top_score)

[[0, 0.0, 0.0], [1, 0.08333333333333333, 0.07692307692307693], [2, 0.13333333333333333, 0.18181818181818182], [3, 0.09615384615384616, 0.4166666666666667], [4, 0.0, 0.0], [5, 0.07692307692307693, 0.3333333333333333], [6, 0.03125, 0.1], [7, 0.1111111111111111, 0.03333333333333333], [8, 0.02702702702702703, 0.09090909090909091], [9, 0.045454545454545456, 0.2], [10, 0.09090909090909091, 0.14285714285714285], [11, 0.16666666666666666, 0.058823529411764705], [12, 0.045454545454545456, 0.02702702702702703], [13, 0.23076923076923078, 0.23076923076923078], [14, 0.020833333333333332, 0.01639344262295082], [15, 0.4, 0.03125], [16, 0.05, 0.23076923076923078], [17, 0.1111111111111111, 0.13333333333333333], [18, 0.07692307692307693, 0.25], [19, 0.06451612903225806, 0.1], [20, 0.1111111111111111, 0.16666666666666666], [21, 0.0, 0.0], [22, 0.0, 0.0], [23, 0.13043478260869565, 0.21428571428571427], [24, 0.0, 0.0], [25, 0.07692307692307693, 0.1], [26, 0.14285714285714285, 0.2], [27, 0.07142857142857142

In [63]:
bleu_top_scores_one_sentence = 0
rouge_top_scores_one_sentence = 0

for elem in scores_top_score:
    rouge_top_scores_one_sentence = rouge_top_scores_one_sentence+elem[1]
    bleu_top_scores_one_sentence = bleu_top_scores_one_sentence+elem[2]
print('ROUGE : '+str(rouge_top_scores_one_sentence))
print('BLEU  : '+str(bleu_top_scores_one_sentence))

ROUGE : 6.705492392982123
BLEU  : 9.346182335910829


<div class="alert alert-block alert-info">

  B) goldstandard summaries = summaries for each book found online at https://www.printfriendly.com/p/g/tp79hs <br>

<div>

In [8]:
#create a gold standard from online summaries
#the online summaries where pasted into the file
with open('book_summaries.txt', 'r') as f:
    words=f.readlines()

filtered_list = list(filter(lambda a: a != 'New Testament\n',filter(lambda a: a != 'Advertisements\n',filter(lambda a: a != '\n', words))))
filtered_list[0]


'Old Testament\n'

In [9]:
concat_list = []
book_counter = 1

for i in range(len(filtered_list)):
    if i % 2 == 0 :
        if i!=0:
            book_nr = book_counter-1
            concat_list.append(['BOOK'+str(book_nr),filtered_list[i]])
        book_counter+=1
#concat_list    

In [10]:
from nltk.corpus import stopwords
import string

stopwords = list(stopwords.words('english'))
newstopwords = ['shalt', 'saith', 'thy','thee', 'thou', 'shall', 'thine', 'ye', 'you', 'whereof', 'didst', 'thereof', 'shouldest', 'unto', 'said', 'upon', 'also', 'abound', 'book64', 'book57', 'us', 'ai', 'book63', 'let', 'hereby', 'hath', 'saying', 'book65', 'doeth', 'say']
stopwords.extend(newstopwords)

diff_gold_summaries = []
for summary in concat_list:
    clean_summary_per_book = []
    single_words  = summary[1].split(' ')
    for word in single_words: 
        if word not in stopwords:
            clean_summary_per_book.append(word)
    clean_summary_per_book = list(dict.fromkeys(clean_summary_per_book))
    diff_gold_summaries.append([summary[0],clean_summary_per_book])

# diff_gold_summaries[0]

online_gold_standard = [[word.replace('“', '').replace('\n','').replace('?','').replace('”','').replace('’s','').lower() for word in elem[1]] for elem in diff_gold_summaries]
online_gold_standard[1]
len(online_gold_standard)

66

In [11]:
#Now we can execute the rouge_bleu function for our topten_words and the gold standard B)
scores_online_gold_standard = []
for n in range(len(online_gold_standard)):
    rouge,bleu = rouge_bleu(topten_words[n], online_gold_standard[n])
    scores_online_gold_standard.append([n,rouge,bleu ])
#scores_online_gold_standard

In [48]:
#scores_online_gold_standard
rouge_scores_top_ten_online_s = 0
bleu_scores_topten_online_s = 0

for elem in scores_online_gold_standard:
    rouge_scores_topten_online_s = rouge_scores_top_ten_online_s+elem[1]
    bleu_scores_topten_online_s = bleu_scores_topten_online_s+elem[2]
print('ROUGE : '+str(rouge_scores_topten_online_s))
print('BLEU  : '+str(bleu_scores_topten_online_s))

ROUGE : 0.0
BLEU  : 5.1999999999999975


In [17]:
#and we can execute rouge_bleu for our top_tfidf_score words and the gold standard B)

scores_top_score_online = []
for n in range(len(gold_summaries_words)):
    #gold_summaries_words[n].pop(0)
    rouge,bleu = rouge_bleu(words_top_tfidf_scores[n], gold_summaries_words[n])
    scores_top_score_online.append([n,rouge,bleu ])
#print(scores_top_score_online)

In [49]:
#scores_online_gold_standard
rouge_top_scores_online_s = 0
bleu_top_scores_online_s = 0

for elem in scores_top_score_online:
    rouge_top_scores_online_s = rouge_top_scores_online_s+elem[1]
    bleu_top_scores_online_s = bleu_top_scores_online_s+elem[2]
print('ROUGE : '+str(rouge_top_scores_online_s))
print('BLEU  : '+str(bleu_top_scores_online_s))

ROUGE : 6.705492392982123
BLEU  : 9.346182335910829


The result of compareLists() cofirms that the set of topten words of our summaries have no overlappings with the relevant words from the online summaries.

<div class="alert alert-block alert-info">

  
  C) goldstandard summaries = summaries for each chapter combined per book, found online at<br>

<div>

In [13]:
with open('bible_chapter_summaries_66.txt', 'r') as f:
    summaries_66=f.readlines()
#summaries_66[0]

In [14]:
import string
len(summaries_66)
type(summaries_66[0])
cleaned_summaries_66 = []
index_counter = 0
for summary in summaries_66:
    new_list = summary.split(' ')
    cleaned_book_summary_words = []
    #remove brackets and unwanted chars from the first and the last element of the list:
    new_first_elem = re.sub('\'','',re.sub('\[','',summary[0]))
    new_last_elem = re.sub('\\n','',re.sub('\'\]','',re.sub('\.','',new_list[-1:][0] )))
    
    new_list.pop(0)
    new_list.insert(0,new_first_elem)
    new_list.pop()
    new_list.append(new_last_elem)
    
    for elem in new_list: 
        if elem not in stopwords:
            cleaned_book_summary_words.append(elem)
    remove_duplicates = list(set(cleaned_book_summary_words))
    remove_punctuation = []
    #remove punctuation marks
    for elem in remove_duplicates:
        translator = str.maketrans('', '', string.punctuation)
        remove_punctuation.append(elem.translate(translator))
            
    #cleaned_summaries_66.append(cleaned_book_summary_words)
    cleaned_summaries_66.append(remove_punctuation)
#    if index_counter ==0:
#        print(type(cleaned_book_summary_words))
#        print(cleaned_book_summary_words[1])
#        print(type(cleaned_summaries_66[0]))
#        print(cleaned_summaries_66[0][1])
#    index_counter += 1
new_list[1]
   

'revelation'

In [53]:

#Now we can execute the rouge_bleu function for our topten_words and the gold standard C)
scores_topten_words_chapter = []
for i in range(len(cleaned_summaries_66)):
    rouge,bleu = rouge_bleu(topten_words[i],cleaned_summaries_66[i])
    scores_topten_words_chapter.append([i, rouge,bleu])

#scores_topten_words_chapter



ROUGE : 1.6444787963130405
BLEU  : 19.300000000000004


In [ ]:
rouge_topten_allchapters_summary = 0
bleu_topten_allchapters_summary = 0

for elem in scores_topten_words_chapter:
    rouge_topten_allchapters_summary = rouge_topten_allchapters_summary+elem[1]
    bleu_topten_allchapters_summary = bleu_topten_allchapters_summary+elem[2]
print('ROUGE : '+str(rouge_topten_allchapters_summary))
print('BLEU  : '+str(bleu_topten_allchapters_summary))

In [56]:
#and we can execute rouge_bleu for our top_tfidf_score words and the gold standard C)
similarity_scores_summary_from_chapter = []
scores_top_score_chapter = []
for i in range(len(cleaned_summaries_66)):
    rouge, bleu= rouge_bleu(words_top_tfidf_scores[i],cleaned_summaries_66[i])
    scores_top_score_chapter.append([i, rouge, bleu])
#print(scores_top_score_chapter)



In [57]:
rouge_top_scores_allchapters_summary = 0
bleu_top_scores_allchapters_summary = 0

for elem in scores_top_score_chapter:
    rouge_top_scores_allchapters_summary = rouge_top_scores_allchapters_summary+elem[1]
    bleu_top_scores_allchapters_summary = bleu_top_scores_allchapters_summary+elem[2]
print('ROUGE : '+str(rouge_top_scores_allchapters_summary))
print('BLEU  : '+str(bleu_top_scores_allchapters_summary))

ROUGE : 1.8044210051515361
BLEU  : 12.162355845115012


<div class="alert alert-block alert-info">4. Examine german summaries:
    </div>

For the german summaries we ran out of time and could therefore not use the second approach of creating summaries based on the overall tf-idf score. Therefore, in the following cells we only compare our summaries created with the first approach to the german gold standard summaries.

In [31]:
#use the words for the german summaries

#open the txt which contains the words from the goldstandard summaries in german:
with open('words_german_goldstandard_summaries.txt', 'r', encoding = 'utf-8') as f:
    words_goldstandard_german=f.readlines()    

#and check the format
print(len(words_goldstandard_german))
print(type(words_goldstandard_german))
print(words_goldstandard_german[0])


66
<class 'list'>
['schoepfung', 'vorfahren', 'einheit', 'geworden', 'dar', 'leben', 'ab', 'foerdert', 'geschichten', 'ist', 'erzaehlungen', 'erzaehlen', 'mosebuches', 'geschichte', 'welt', 'anfang', 'berichten', 'enden', 'menschheit', 'menschenpaar', 'gut', 'gott', 'einsetzt', 'abrahams', 'wollen', 'orientiert', 'einst', 'hindert', 'war', 'fuehrt', 'darin', 'heute', 'vielmehr', 'erste', 'erleben', 'turmbau', 'kapitel', 'erkennt', 'mythen', 'gen', 'warum', 'ersten', 'unserer', 'damit', 'urgeschichte', 'genannten', 'boese', 'stellen', 'dadurch', '111', 'heben', 'gesamten', 'folgenden', 'aehnlich', 'babel']



In [32]:

#open the txt which contains the words from the summaries in german generated with topten tf idf:
with open('words_topten_german_summaries.txt', 'r', encoding = 'utf-8') as f:
    words_topten_german=f.readlines()    


print(len(words_topten_german))
print(type(words_topten_german))
print(words_topten_german[0])



66
<class 'list'>
['sprach', 'jakob', 'abram', 'joseph', 'laban', 'abraham', 'pharao', 'herr', 'isaak']



In [58]:

#Now we can execute the rouge_bleu function for our topten_words in german and and the gold standard respectively
scores_topten_words_german = []
for i in range(len(words_goldstandard_german)):
    rouge,bleu = rouge_bleu(words_topten_german[i],words_goldstandard_german[i])
    scores_topten_words_german.append([i, rouge,bleu])
#scores_topten_words_german

In [59]:
rouge_topten_one_sentence_german = 0
bleu_topten_one_sentence_german = 0

for elem in scores_topten_words_german:
    rouge_topten_one_sentence_german = rouge_topten_one_sentence_german+elem[1]
    bleu_topten_one_sentence_german = bleu_topten_one_sentence_german+elem[2]
print('ROUGE : '+str(rouge_topten_one_sentence_german))
print('BLEU  : '+str(bleu_topten_one_sentence_german))

ROUGE : 6.95324916914873
BLEU  : 65.8121662307199


<div class="alert alert-block alert-info">5. First steps of attempt for using wordNets
    </div>

In the following cells we started to examine the word relatedness of our english text where we have very low ROUGE/BLEU scores. The idea is to find that even though the scores are low, there might be some common hypernyms of the words from our goldstandards and the words from summaries generated by us. 

In [60]:
#first, import wordnet
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [126]:
#recap the structure of synsets by checking the synsets of "created"
#created is one of the words which was not found to be common in the summaries generated by us and the gold standard
wn.synsets('created')

[Synset('make.v.03'),
 Synset('create.v.02'),
 Synset('create.v.03'),
 Synset('create.v.04'),
 Synset('create.v.05'),
 Synset('produce.v.02')]

In [109]:
wn.synset('create.v.02')

Synset('create.v.02')

In [110]:
# STUDENT ANSWER
def have_hypernym(w1,w2):
    i,j = 0,0
    w1_all_synsets = wn.synsets(w1)
    w2_all_synsets = wn.synsets(w2)
    
    word_w1,word_w2 = [],[]
    
    while i < len(w1_all_synsets):
            word_w1 = w1_all_synsets[i].hypernyms()
            for e in word_w1:
                while j < len(w2_all_synsets):
                    word_w2 = w2_all_synsets[j].hypernyms()
                    for f in word_w2:
                        if f == e:
                            print(str(f))
                            return True
                    j+=1
            i+=1
    
    return False

In [111]:
#test the function
have_hypernym('apple','pear')

Synset('edible_fruit.n.01')


True

the "have_hypernym"-funtion takes strings as input. Therefore it should work on the wordlists that we have above. But it seems unnecessary to compare words that occur in both lists that are compared. Therefore, the idea is to create two lists of words that are not common for every comparisson process ( i. e. calling the "rouge_bleu"-function) which can be obtained using the "compareLists"-function. One list would hold all words from our summaries which are not in the gold standard list and the other vice versa. We could then compare each word from both list with another and if we find a hypernym e.g. on the first level (=both words have the same direct hypernym), the word gets scored. 